In [106]:
import torch
import torchvision
import torchvision.transforms as pth_transforms
from sklearn.neighbors import KNeighborsClassifier
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler
import torch.utils.data as Data
from google.colab import drive
drive.mount('/content/drive')

import sys
# Append the directory to your python path using sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
def get_instance_noisy_label(n, dataset, labels, num_classes, feature_size, norm_std, seed):
    # n -> noise_rate
    # dataset -> mnist, cifar10 # not train_loader
    # labels -> labels (targets)
    # label_num -> class number
    # feature_size -> the size of input images (e.g. 28*28)
    # norm_std -> default 0.1
    # seed -> random_seed
    print("building dataset...")
    label_num = num_classes
    np.random.seed(int(seed))
    torch.manual_seed(int(seed))
    torch.cuda.manual_seed(int(seed))

    P = []
    flip_distribution = stats.truncnorm((0 - n) / norm_std, (1 - n) / norm_std, loc=n, scale=norm_std)
    flip_rate = flip_distribution.rvs(labels.shape[0])

    if isinstance(labels, list):
        labels = torch.FloatTensor(labels)
    labels = labels.cuda()

    W = np.random.randn(label_num, feature_size, label_num)


    W = torch.FloatTensor(W).cuda()
    for i, (x, y) in enumerate(dataset):
        # 1*m *  m*10 = 1*10
        x = x.cuda()
        A = x.view(1, -1).mm(W[y]).squeeze(0)
        A[y] = -inf
        A = flip_rate[i] * F.softmax(A, dim=0)
        A[y] += 1 - flip_rate[i]
        P.append(A)
    P = torch.stack(P, 0).cpu().numpy()
    l = [i for i in range(label_num)]
    new_label = [np.random.choice(l, p=P[i]) for i in range(labels.shape[0])]
    record = [[0 for _ in range(label_num)] for i in range(label_num)]

    for a, b in zip(labels, new_label):
        a, b = int(a), int(b)
        record[a][b] += 1


    pidx = np.random.choice(range(P.shape[0]), 1000)
    cnt = 0
    for i in range(1000):
        if labels[pidx[i]] == 0:
            a = P[pidx[i], :]
            cnt += 1
        if cnt >= 10:
            break
    return np.array(new_label)
def transform_target(label):
    label = np.array(label)
    target = torch.from_numpy(label).long()
    return target

def data_split(data, targets, split_percentage, seed=1):

    num_samples = int(targets.shape[0])
    np.random.seed(int(seed))
    train_set_index = np.random.choice(num_samples, int(num_samples*split_percentage), replace=False)
    index = np.arange(data.shape[0])
    val_set_index = np.delete(index, train_set_index)
    train_set, val_set = data[train_set_index, :], data[val_set_index, :]
    train_labels, val_labels = targets[train_set_index], targets[val_set_index]

    return train_set, val_set, train_labels, val_labels


class cifar10_dataset(Data.Dataset):
    def __init__(self, train=True, transform=None, target_transform=None, noise_rate=0.2, split_percentage=0.9, seed=1, num_classes=10, feature_size=3*32*32, norm_std=0.1):

        self.transform = transform
        self.target_transform = target_transform
        self.train = train

        original_images = np.load('/content/drive/MyDrive/Colab Notebooks/data/cifar10/train_images.npy')
        original_labels = np.load('/content/drive/MyDrive/Colab Notebooks/data/cifar10/train_images.npy')
        data = torch.from_numpy(original_images).float()
        targets = torch.from_numpy(original_labels)

        dataset = zip(data, targets)
        new_labels = get_instance_noisy_label(noise_rate, dataset, targets, num_classes, feature_size, norm_std, seed)


        self.train_data, self.val_data, self.train_labels, self.val_labels = data_split(original_images, new_labels, split_percentage,seed)
        if self.train:
            self.train_data = self.train_data.reshape((-1,3,32,32))
            self.train_data = self.train_data.transpose((0, 2, 3, 1))
            print(self.train_data.shape)

        else:
            self.val_data = self.val_data.reshape((-1, 3,32,32))
            self.val_data = self.val_data.transpose((0, 2, 3, 1))

    def __getitem__(self, index):

        if self.train:
            img, label = self.train_data[index], self.train_labels[index]

        else:
            img, label = self.val_data[index], self.val_labels[index]

        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            label = self.target_transform(label)

        return img, label
    def __len__(self):

        if self.train:
            return len(self.train_data)

        else:
            return len(self.val_data)

In [109]:
num_classes = 10
feature_size = 3 * 32 * 32
n_epoch_1, n_epoch_2, n_epoch_3 = 5, 50, 50
dim = 512
basis = 20
iteration_nmf = 10
train_dataset = cifar10_dataset(True,
                                    transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010)),
                                    ]),
                                    target_transform=transform_target)


building dataset...


<ipython-input-107-1c252bcca995>:30: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  A = x.view(1, -1).mm(W[y]).squeeze(0)


IndexError: The shape of the mask [3072] at index 0 does not match the shape of the indexed tensor [10, 3072, 10] at index 0

In [ ]:
torch.hub.list('facebookresearch/dino:main')

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


['dino_resnet50',
 'dino_vitb16',
 'dino_vitb8',
 'dino_vits16',
 'dino_vits8',
 'dino_xcit_medium_24_p16',
 'dino_xcit_medium_24_p8',
 'dino_xcit_small_12_p16',
 'dino_xcit_small_12_p8',
 'resnet50']

In [ ]:
torch.hub.help('facebookresearch/dino:main', 'dino_vitb16')

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


'\n    ViT-Base/16x16 pre-trained with DINO.\n    Achieves 76.1% top-1 accuracy on ImageNet with k-NN classification.\n    '

In [ ]:
model = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16', pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    model.cuda()
model.eval()

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_vitbase16_pretrain/dino_vitbase16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_vitbase16_pretrain.pth
100%|██████████| 327M/327M [00:07<00:00, 48.6MB/s]


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

In [81]:
import numpy as np
import torch
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from torch import nn
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
class KNN():
    def __init__(self, model, k, device):
        super(KNN, self).__init__()
        self.k = k
        self.device = device
        self.model = model.to(device)
        self.model.eval()

    def extract_features(self, loader):
        """
        Infer/Extract features from a trained model
        Args:
            loader: train or test loader
        Returns: 3 tensors of all:  input_images, features , labels
        """
        x_lst = []
        features = []
        label_lst = []

        with torch.no_grad():
            for input_tensor, label in loader:
                h = self.model(input_tensor.to(self.device))
                features.append(h)
                x_lst.append(input_tensor)
                label_lst.append(label)

            x_total = torch.stack(x_lst)
            h_total = torch.stack(features)
            label_total = torch.stack(label_lst)

            return x_total, h_total, label_total

    def knn(self, features, labels, k=1):
        """
        Evaluating knn accuracy in feature space.
        Calculates only top-1 accuracy (returns 0 for top-5)
        Args:
            features: [... , dataset_size, feat_dim]
            labels: [... , dataset_size]
            k: nearest neighbours
        Returns: train accuracy, or train and test acc
        """
        feature_dim = features.shape[-1]
        with torch.no_grad():
            features_np = features.cpu().view(-1, feature_dim).numpy()
            labels_np = labels.cpu().view(-1).numpy()
            # fit
            self.cls = KNeighborsClassifier(k, metric="cosine").fit(features_np, labels_np)
            acc = self.eval(features, labels)

        return acc

    def eval(self, features, labels):
      feature_dim = features.shape[-1]
      features = features.cpu().view(-1, feature_dim).numpy()
      labels = labels.cpu().view(-1).numpy()
      acc = 100 * np.mean(cross_val_score(self.cls, features, labels))
      predicted_labels = self.cls.predict(features)
      classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
      num_classes=10
      noise_conf_matrix = confusion_matrix(labels, predicted_labels)
      noise_transitions = noise_conf_matrix / noise_conf_matrix.sum(axis=1, keepdims=True)
      plt.figure(figsize=(12, 4))
      plt.subplot(1, 2, 2)
      sns.heatmap(noise_transitions, annot=True, cmap="Reds", fmt=".2f", cbar=False,
                  xticklabels=[f'{classes[i]}' for i in range(num_classes)],
                  yticklabels=[f'{classes[i]}' for i in range(num_classes)])
      plt.title('Noise Label Transition Matrix')
      return acc

    def _find_best_indices(self, h_query, h_ref):
        h_query = h_query / h_query.norm(dim=1).view(-1, 1)
        h_ref = h_ref / h_ref.norm(dim=1).view(-1, 1)
        scores = torch.matmul(h_query, h_ref.t())  # [query_bs, ref_bs]
        score, indices = scores.topk(1, dim=1)  # select top k best
        return score, indices

    def fit(self, test_loader=None):
        with torch.no_grad():
          x_test, h_test, l_test = self.extract_features(test_loader)
          test_acc = self.knn(h_test, l_test, k=self.k)
          return(test_acc)

In [89]:
ssl_evaluator = KNN(model=model, k=1, device='cuda')
ssl_evaluator.fit(test_dataset)

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from PIL import Image

def plot_result(query_image, retrieved_indices, dataset):
    # Plot the query image
    plt.subplot(2, 3, 1)
    plt.imshow(query_image)
    plt.title("Query Image")

    # Plot the top 5 retrieved images
    for i, idx in enumerate(retrieved_indices):
        image_path = dataset.image_paths[idx]
        image = Image.open(image_path)

        plt.subplot(2, 3, i + 2)
        plt.imshow(image)
        plt.title(f"Retrieved Image {i+1}")

    plt.tight_layout()
    plt.show()

# Example usage:
# Assuming you have an instance of ImageRetrievalKNN named 'image_retrieval_knn'
query_image, query_label = test_dataset[0]  # Replace with your query image
retrieved_indices = image_retrieval_knn.image_retrieval(query_image)

# Plot the result
plot_result(query_image, retrieved_indices, test_dataset)


In [ ]:
import torch
from tqdm.notebook import tqdm

features = []
labels = []  # Create a list to store labels
for samples, index in tqdm(data_loader_train, desc='Batches'):
    samples = samples.cuda(non_blocking=True)
    #index = index.cuda(non_blocking=True)  # We don't need the index on GPU
    with torch.no_grad():
        #feature_batch = model(samples).clone()
        # Let's not keep the features around on the GPU, also cloning keeps
        # reference to the computational graph which could cause memory issues
        # outside of no_grad().
        # detach() decouples the computation from the graph which created it.
        # Since we're in a no_grad() context, that probably doesn't matter since
        # the results are likely not chached.
        feature_batch = model(samples)
    features.append(feature_batch)
    labels.append(index)

# Combine the feature batches into a single tensor
features_tensor = torch.cat(features, dim=0)
labels_tensor = torch.cat(labels, dim=0)

print(f"Storing features into tensor of shape {features_tensor.shape}")
print(f"Storing labels into tensor of shape {labels_tensor.shape}")


In [ ]:
import numpy as np
from sklearn.manifold import TSNE

features_array = features_tensor.numpy()
labels_array = labels_tensor.cpu().numpy()

tsne = TSNE(n_components=3)
reduced_features = tsne.fit_transform(features_array)

In [ ]:
labels_array

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 12))
plt.scatter(reduced_features[:, 0], reduced_features[:, 1], s=2, alpha=.5, c=labels_array, cmap='tab10')
plt.title("t-SNE Clustering of Training Features")
#plt.colorbar()
plt.legend()
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(reduced_features, labels_array)